In [ ]:
# imports
import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, Flatten, Dense, Lambda, Reshape, MaxPooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import save_model
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import os
os.sys.path

import pickle
import joblib
# added so that cv2 gets installed in kernal
 import sys
 !{sys.executable} -m pip install opencv-python
# commented the above code, it started working, idk why
# if code not working try uncommenting the above
import cv2
import random
import glob

# Data Processing

## Getting data from Folder

In [ ]:
os.chdir("images")

data_exists = os.path.exists("train_data.z")

IMG_SIZE = 128

if not data_exists:
    data = []
    path = os.getcwd()
    print(path)
    
    def create_data():
        count = 0
        for folder in os.listdir(path):
            if "train_data" in folder:
                continue
            print("FOLDER: ", folder)
            for filename in os.listdir(path + "/" + folder):
                if (".NEF" in filename):
                    temp_path = path + "/" + folder + "/" + filename
                    try:
                        img_array = cv2.imread(temp_path)
                        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
                        img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                        data.append(img_array)
                        count += 1
                    except Exception as e:
                        pass
                    
    create_data()
else:
    print("train_data.z exists, loading file...")
    train_data = joblib.load("train_data.z")
    

In [ ]:
if not data_exists:
    # can explore what the dataset has
    plt.imshow(data[100])

## Making Data Readable for Model

In [ ]:
if not data_exists:
    #Resizing the data

    train_data = data
    train_data = np.array(train_data).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    train_data = train_data/255.0

## Storing Processed Data

In [ ]:
if not data_exists:
    joblib.dump(train_data, "train_data.z")

## Info On Picture Data

In [ ]:
# General Info:
print("Number of Images:", len(train_data))
print("Shape of each data item:", train_data[0].shape)
print(train_data[0][10])
# First couple of items should be:
'''
[[0.01568627]
 [0.01568627]
 [0.01960784]
 [0.01568627]
 [0.01568627]
...
'''

In [ ]:
# look at some images
def plotImages(images_arr):
    fig, axes =  plt.subplots(1, 10, figsize=(30,30))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(np.squeeze(img)) # added squeeze to make it work
        ax.axis('off')
    plt.tight_layout()
    plt.show()
    
plotImages(train_data)

In [ ]:
latent_dim = 2048

input_img = Input(shape=input_shape, name='encoder_input')
x = Conv2D(64, 3, padding='same', activation='relu')(input_img)
x = MaxPooling2D((2,2), padding='same')(x)
x = Dropout(0.2)(x)
x = Conv2D(128, 3, padding='same', activation='relu')(x)
x = MaxPooling2D((2,2),padding='same')(x)
x = Dropout(0.2)(x)
x = Conv2D()